In [19]:
import pandas as pd
import logging
import sqlite3
import time
import email
from email.header import decode_header
import imaplib

In [20]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("email_processor.log"),
    ]
)

In [21]:
def retry(func, retries=3, delay=5, *args, **kwargs):
    """Retries a function upon failure."""
    for attempt in range(retries):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            logging.error(f"Attempt {attempt + 1} failed: {e}")
            time.sleep(delay)
    raise Exception("All retries failed")

In [22]:
# Setup SQLite database
def setup_database():
    conn = sqlite3.connect("emails.db")
    cursor = conn.cursor()
    cursor.execute(
        """
        CREATE TABLE IF NOT EXISTS emails (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            sender TEXT,
            subject TEXT,
            timestamp TEXT
        )
        """
    )
    conn.commit()
    conn.close()

In [23]:
# Connect to email
def connect_to_email(username, password):
    try:
        mail = imaplib.IMAP4_SSL("imap.gmail.com")
        mail.login(username, password)
        return mail
    except Exception as e:
        logging.error(f"Error connecting to email: {e}")
        raise

In [24]:
# Process emails
def process_emails(mail):
    try:
        mail.select("inbox")
        status, messages = mail.search(None, "UNSEEN")
        if status != "OK":
            logging.warning(f"Error searching for emails: {status}")
            return
        email_ids = messages[0].split()
        logging.info(f"Found {len(email_ids)} unread emails")

        conn = sqlite3.connect("emails.db")
        cursor = conn.cursor()

        for email_id in email_ids:
            res, msg = mail.fetch(email_id, "(RFC822)")
            for response in msg:
                if isinstance(response, tuple):
                    msg = email.message_from_bytes(response[1])

                    sender = msg.get("From", "Unknown")
                    subject = msg.get("Subject", "No Subject")
                    timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())

                    try:
                        subject, encoding = decode_header(subject)[0]
                        if isinstance(subject, bytes):
                            subject = subject.decode(encoding or "utf-8")
                    except Exception:
                        pass

                    cursor.execute(
                        "INSERT INTO emails (sender, subject, timestamp) VALUES (?, ?, ?)",
                        (sender, subject, timestamp),
                    )
                    logging.info(f"Processed email: Sender={sender}, Subject={subject}, Timestamp={timestamp}")
                    mail.store(email_id, '+FLAGS', '\\Seen')
                    #logging.getLogger().handlers[0].flush()  # Explicit flush

        conn.commit()
        conn.close()
        logging.info("Emails processed successfully.")
    except Exception as e:
        logging.error(f"Error processing emails: {e}")

In [25]:
def main():
    setup_database()

    # credentials
    username = "your_email_here" # Use a Gmail account
    password = "your_password_here" # use your password or generate an App password

    try:
        # Retry connecting to the email server
        mail = retry(connect_to_email, retries=3, delay=5, username=username, password=password)

        # Retry processing emails
        retry(process_emails, retries=3, delay=5, mail=mail)

        mail.logout()
    except Exception as e:
        logging.error(f"Error in main: {e}")

In [26]:
if __name__ == "__main__":
    main()

In [27]:
# Display SQLite data
import pandas as pd
conn = sqlite3.connect("emails.db")
df = pd.read_sql_query("SELECT * FROM emails", conn)
conn.close()
df

,id,sender,subject,timestamp
0,1,Saumya Dwivedi <saumyadwivedi703@gmail.com>,Who Let the,2024-12-13 05:50:34
1,2,Saumya Dwivedi <saumyadwivedi703@gmail.com>,Dogs out,2024-12-13 05:50:35
2,3,Saumya Dwivedi <saumyadwivedi703@gmail.com>,Never gonna,2024-12-13 06:07:41
3,4,Saumya Dwivedi <saumyadwivedi703@gmail.com>,Give you up,2024-12-13 06:07:42


In [28]:
# Display logs
with open("email_processor.log", "r") as log_file:
    logs = log_file.read()
    print(logs)

2024-12-13 10:53:08,461 - INFO - Found 0 unread emails
2024-12-13 10:53:08,464 - INFO - Emails processed successfully.
2024-12-13 11:20:34,074 - INFO - Found 2 unread emails
2024-12-13 11:20:34,863 - INFO - Processed email: Sender=Saumya Dwivedi <saumyadwivedi703@gmail.com>, Subject=Who Let the, Timestamp=2024-12-13 05:50:34
2024-12-13 11:20:35,303 - INFO - Processed email: Sender=Saumya Dwivedi <saumyadwivedi703@gmail.com>, Subject=Dogs out, Timestamp=2024-12-13 05:50:35
2024-12-13 11:20:35,403 - INFO - Emails processed successfully.
2024-12-13 11:37:40,602 - INFO - Found 2 unread emails
2024-12-13 11:37:41,150 - INFO - Processed email: Sender=Saumya Dwivedi <saumyadwivedi703@gmail.com>, Subject=Never gonna, Timestamp=2024-12-13 06:07:41
2024-12-13 11:37:42,317 - INFO - Processed email: Sender=Saumya Dwivedi <saumyadwivedi703@gmail.com>, Subject=Give you up, Timestamp=2024-12-13 06:07:42
2024-12-13 11:37:42,925 - INFO - Emails processed successfully.

